In [1]:
import numpy as np
import librosa
import tensorflow as tf
from tensorflow.keras import layers, models
import os

In [2]:
def parse_note_from_filename(filename):
    parts = filename.split('_')
    
    if len(parts) > 0:
        return parts[0]
    else:
        return 'unknown'

In [3]:
import numpy as np
import librosa
import os

# Set a fixed length for all audio samples (you can adjust this as needed)
fixed_length = 44100  # for example, 1 second at 44.1 kHz
fft = librosa.feature.mfcc

audio_samples = []
musical_notes = []

# Define a function to load and preprocess an audio file
def preprocess_audio(audio_file_path, target_length=fixed_length):
    audio, sr = librosa.load(audio_file_path, sr=None)
    
    # Ensure that all audio samples have the same length
    if len(audio) < target_length:
        audio = np.pad(audio, (0, target_length - len(audio)))
    elif len(audio) > target_length:
        audio = audio[:target_length]
    
    # Extract features (FFT in this case)
    fft_result = np.fft.fft(audio)
    magnitudes = np.abs(fft_result)
    
    return magnitudes

folder_path = 'piano_triads'

for filename in os.listdir(folder_path):
    if filename.endswith('.wav'):
        file_path = os.path.join(folder_path, filename)
        
        # Preprocess the audio and extract features
        features = preprocess_audio(file_path)
        
        note = parse_note_from_filename(filename)
        
        audio_samples.append(features)
        musical_notes.append(note)

X = np.array(audio_samples)
y = np.array(musical_notes)

# Now, X contains the preprocessed audio features (FFT magnitudes) and y contains the corresponding musical notes.

In [4]:
def extract_features(audio_samples):
    features = []
    for audio in audio_samples:
        # Compute MFCC features (you can experiment with other feature extraction methods)
        fft_result = librosa.feature.fft(audio, sr=44100, n_mfcc=13)
        features.append(fft_result)
    return features

In [5]:
# features = extract_features(audio_samples)

# mean_magnitudes = [np.mean(feature) for feature in features]

# import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 6))
# plt.plot(mean_magnitudes)
# plt.xlabel('Sample Index')
# plt.ylabel('Mean Magnitude')
# plt.title('Mean Magnitude of FFT Coefficients for Audio Samples')
# plt.grid()
# plt.show()


In [6]:
# import matplotlib.pyplot as plt

# num_samples_to_visualize = 50

# for i in range(num_samples_to_visualize):
#     plt.figure(figsize=(7, 4))
#     plt.specgram(X[i], Fs=sr, cmap='inferno')

#     filename = os.listdir(folder_path)[i]
    
#     plt.title(f'Spectrogram of {filename}')
#     plt.xlabel('Time (s)')
#     plt.ylabel('Frequency (Hz)')
#     plt.colorbar(format='%+2.0f dB')
#     plt.tight_layout()
#     plt.show()


In [7]:
# #amplitude vs frequency

# import matplotlib.pyplot as plt
# import numpy as np

# num_samples_to_visualize = 50

# for i in range(num_samples_to_visualize):
#     plt.figure(figsize=(7, 4))
    
#     fft_result = np.fft.fft(X[i])
    
#     frequencies = np.fft.fftfreq(len(fft_result), 1/sr)
#     amplitudes = np.abs(fft_result)

#     plt.plot(frequencies[:len(frequencies)//2], amplitudes[:len(amplitudes)//2])  
#     filename = os.listdir(folder_path)[i]
    
#     plt.title(f'Amplitude vs. Frequency of {filename}')
#     plt.xlabel('Frequency (Hz)')
#     plt.ylabel('Amplitude')
#     plt.grid()
#     plt.tight_layout()
#     plt.show()


In [8]:
# # waveform plot
# import librosa.display

# num_samples_to_visualize = 40

# for i in range(num_samples_to_visualize):
#     plt.figure(figsize=(7, 4))
#     plt.plot(np.arange(len(X[i])) / sr, X[i])
#     filename = os.listdir(folder_path)[i]

#     plt.title(f'Waveform of Audio Sample {filename}')
#     plt.xlabel('Time (s)')
#     plt.ylabel('Amplitude')
#     plt.tight_layout()
#     plt.show()

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder


In [10]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Encode the labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# from sklearn.preprocessing import OneHotEncoder

# # Sample labels
# y_train = ["C", "Cs", "D", "Eb", "E", "F", "Fs", "G", "Gs", "A", "Bb", "B"]

# # Initialize the OneHotEncoder
# one_hot_encoder = OneHotEncoder(sparse=False)

# # Reshape and fit-transform the labels
# y_train_encoded = one_hot_encoder.fit_transform(np.array(y_train).reshape(-1, 1))

# # Now, y_train_encoded contains the one-hot encoded labels

# # Inverse transformation (decoding)
# y_train_decoded = one_hot_encoder.inverse_transform(y_train_encoded)

# y_test_encoded = one_hot_encoder.transform(np.array(y_test).reshape(-1, 1))



In [11]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

# Define musical note names
NOTE_NAMES = ["C", "Cs", "D", "Eb", "E", "F", "Fs", "G", "Gs", "A", "Bb", "B"]

from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping

# Define a learning rate schedule function
def lr_schedule(epoch):
    initial_learning_rate = 0.001  # Set your initial learning rate here
    decay = 0.95  # Set the decay rate
    if epoch > 5:  # Adjust this condition based on when you want to start reducing the learning rate
        return initial_learning_rate * (decay ** (epoch - 5))
    else:
        return initial_learning_rate

# Create a learning rate scheduler callback
lr_scheduler = LearningRateScheduler(lr_schedule)

# Create a neural network model
# Define the model
model = Sequential([
    tf.keras.layers.Input(shape=X_train[0].shape),  # Input shape is determined by the chosen feature extraction method
    BatchNormalization(),
    Dense(256, activation='relu'),  # Increased the number of units in the first hidden layer
    Dropout(0.4),
    Dense(128, activation='relu'),  # Increased the number of units in the second hidden layer
    Dense(len(np.unique(y_train_encoded)), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [12]:
# Check the shapes of your data
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train_encoded shape:", y_train_encoded.shape)
print("y_test_encoded shape:", y_test_encoded.shape)

# If there's an inconsistency, check your data split
# Ensure that you are using the same random_state in train_test_split for reproducibility.
# Make sure that X and y are correctly matched before the split.
# Ensure that the test_size is reasonable and doesn't result in an extreme data split.


X_train shape: (346, 44100)
X_test shape: (87, 44100)
y_train_encoded shape: (346,)
y_test_encoded shape: (87,)


In [13]:
# Train your model using X_train_augmented
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

history = model.fit(X_train, y_train_encoded, epochs=50, batch_size=32,
                    validation_data=(X_test, y_test_encoded),
                    callbacks=[lr_scheduler])

Epoch 1/50
11/11 [==============================] - 4s 182ms/step - loss: 14.2181 - accuracy: 0.1358 - val_loss: 13.0858 - val_accuracy: 0.2874 - lr: 0.0010
Epoch 2/50
11/11 [==============================] - 1s 127ms/step - loss: 11.1118 - accuracy: 0.3931 - val_loss: 8.0279 - val_accuracy: 0.4023 - lr: 0.0010
Epoch 3/50
11/11 [==============================] - 1s 131ms/step - loss: 9.0279 - accuracy: 0.4798 - val_loss: 5.2419 - val_accuracy: 0.5057 - lr: 0.0010
Epoch 4/50
11/11 [==============================] - 1s 121ms/step - loss: 6.2676 - accuracy: 0.5549 - val_loss: 4.1314 - val_accuracy: 0.5632 - lr: 0.0010
Epoch 5/50
11/11 [==============================] - 1s 123ms/step - loss: 6.1114 - accuracy: 0.6474 - val_loss: 3.1966 - val_accuracy: 0.6322 - lr: 0.0010
Epoch 6/50
11/11 [==============================] - 1s 124ms/step - loss: 4.8330 - accuracy: 0.6908 - val_loss: 2.3227 - val_accuracy: 0.6322 - lr: 0.0010
Epoch 7/50
11/11 [==============================] - 1s 118ms/step -

In [14]:
# Evaluate the model using encoded labels
test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


3/3 [==============================] - 0s 13ms/step - loss: 1.5641 - accuracy: 0.7701
Test Loss: 1.5641, Test Accuracy: 0.7701


In [15]:
# Make predictions
predictions = model.predict(X_test)

3/3 [==============================] - 0s 18ms/step


In [16]:
# !pip install matplotlib scipy plotly


In [17]:
import numpy as np
import matplotlib.pyplot as plt

def plot_fft(p, xf, fs, notes):
    plt.figure(figsize=(12, 6))
    plt.title("Frequency Spectrum")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Magnitude")
    plt.plot(xf, p)

    for note in notes:
        plt.annotate(note[1], (note[0] + 10, note[2]), fontsize=12, ha='center', va='bottom')

    plt.grid()
    plt.show()

def extract_sample(audio, frame_number, frame_offset):
    end = frame_number * frame_offset
    begin = int(end - FFT_WINDOW_SIZE)

    if end == 0:
        return np.zeros((np.abs(begin)), dtype=float)
    elif begin < 0:
        return np.concatenate([np.zeros((np.abs(begin)), dtype=float), audio[0:end]])
    else:
        return audio[begin:end]

def find_top_notes(fft, num, xf):
    if np.max(fft.real) < 0.001:
        return []

    lst = [x for x in enumerate(fft.real)]
    lst = sorted(lst, key=lambda x: x[1], reverse=True)

    idx = 0
    found = []
    found_note = set()
    while (idx < len(lst)) and (len(found) < num):
        f = xf[lst[idx][0]]
        y = lst[idx][1]
        n = freq_to_number(f)
        n0 = int(round(n))
        name = note_name(n0)

        if name not in found_note:
            found_note.add(name)
            s = [f, note_name(n0), y]
            found.append(s)
        idx += 1

    return found

def freq_to_number(f): return 69 + 12 * np.log2(f / 440.0)
def note_name(n): return NOTE_NAMES[n % 12]


In [18]:
# import numpy as np
# import matplotlib.pyplot as plt

# def plot_fft(magnitudes, sample_rate, notes, xlim=None):
#     fft_values = np.fft.fftfreq(len(magnitudes), 1.0 / sample_rate)
# #
#     plt.figure(figsize=(12, 6))
#     plt.title("Frequency Spectrum")
#     plt.xlabel("Frequency (Hz)")
#     plt.ylabel("Magnitude")
#     plt.plot(fft_values, magnitudes)
    
#     if xlim:
#         plt.xlim(xlim)
    
#     for note in notes:
#         plt.annotate(note[1], (note[0] + 10, note[2]), fontsize=12, ha='center', va='bottom')
    
#     plt.grid()
#     plt.show()

# # Example: Replace this with your actual audio_samples and sample_rate
# # Generate random audio data for demonstration purposes
# sample_rate = 44100
# duration = 5  # 5 seconds of audio
# num_samples = int(sample_rate * duration)
# audio_samples = np.random.randn(num_samples)

# # Extract features and notes (replace with your actual audio and notes)
# features = extract_features([audio_samples])
# num_notes = 5  # Adjust the number of top notes you want to display

# for feature in features:
#     notes = find_top_notes(feature, num_notes, fft_values)  # Replace with your notes
#     plot_fft(feature, sample_rate, notes, xlim=(0, 1000))  # Adjust frequency range if needed


In [19]:
# model.save("mfcc_model2")

In [105]:
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import LabelEncoder

# Define musical note names
NOTE_NAMES = ["C", "Cs", "D", "Eb", "E", "F", "Fs", "G", "Gs", "A", "Bb", "B"]

# Load your trained model
model = tf.keras.models.load_model('trial1')  # Replace 'your_model_path' with the actual path to your trained model

def extract_features(audio):
    # Applying FFT
    fft_result = np.fft.fft(audio)
    magnitudes = np.abs(fft_result)
    return magnitudes

def predict_note(audio_file_path, model):
    audio, _ = librosa.load(audio_file_path, sr=None)
    features = extract_features(audio)
    
    if len(features) < X_train.shape[1]:
        features = np.pad(features, (0, X_train.shape[1] - len(features)))
    elif len(features) > X_train.shape[1]:
        features = features[:X_train.shape[1]]
    
    features = np.array([features])  
    
    prediction = model.predict(features)
    
    predicted_note_index = np.argmax(prediction)
    predicted_note = NOTE_NAMES[predicted_note_index]
    
    return predicted_note

audio_file_path = '88-piano_new/G#_1.wav'
predicted_note = predict_note(audio_file_path, model)
print(f"Predicted Note: {predicted_note}")


1/1 [==============================] - 0s 92ms/step
Predicted Note: G


###### 